# Projek Practicum 1: Membandingkan Preferensi Musik di Springfield dan Shelbyville

Dalam proyek kali ini, Kita akan membandingkan preferensi musik pengguna di kota Springfield dan Shelbyville. Kita akan mempelajari data untuk menguji hipotesis dan membandingkan perilaku pengguna di kedua kota ini.

# Konten <a id='back'></a>

* [Tahap 1. Ikhtisar Data](#data_review)
* [Tahap 2. Pra-pemrosesan data](#data_preprocessing)
    * [2.1 Gaya Penulisan Judul](#header_style)
    * [2.2 Nilai-Nilai yang Hilang](#missing_values)
    * [2.3 Duplikat](#duplicates)
* [Tahap 3. Pengujian Hipotesis](#hypotheses)
    * [3.1 Hipotesis 1: Aktivitas pengguna di kedua kota](#activity)
    * [3.2 Hipotesis 2: Preferensi musik pada hari Senin dan Jumat](#week)
    * [3.3 Hipotesis 3: Preferensi genre di kota Springfield dan Shelbyville](#genre)
* [Kesimpulan](#end)

## Tahap 1. Ikhtisar Data <a id='data_review'></a>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/datasets/music_project_en.csv')

Pertama, kita akan melihat informasi umum tentang tabel dengan satu perintah:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63736 non-null  object
 2   artist    57512 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Tampilkan 10 baris tabel pertamanya:

In [3]:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Shelbyville,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Springfield,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Shelbyville,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Shelbyville,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Springfield,08:34:34,Monday
5,842029A1,Chains,Obladaet,rusrap,Shelbyville,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Springfield,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Springfield,20:47:49,Wednesday
8,8FA1D3BE,L’estate,Julia Dalia,ruspop,Springfield,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Shelbyville,21:20:49,Wednesday


Terdapat pula jumlah nilai yang berbeda antar kolom. Semuanya menyimpan tipe data yang sama, yaitu: `object`.

Berdasarkan dokumentasi:
- `'userID'` — ID pengguna
- `'Track'` — judul trek lagu
- `'artist'` — nama artis
- `'genre'`
- `'City'` — kota tempat pengguna berada
- `'time'` — lama waktu lagu tersebut dimainkan
- `'Day'` — hari dalam seminggu

Kita dapat melihat tiga masalah dengan gaya penulisan nama kolom:
* Beberapa nama ditulis dalam huruf besar, beberapa dalam huruf kecil.
* Beberapa nama menggunakan spasi.
* Jumlah nilai kolom berbeda. Hal ini menandakan bahwa data yang kita miliki mengandung nilai yang hilang.


## Tahap 2. Pra-pemrosesan Data <a id='data_preprocessing'></a>

### Gaya Penulisan Judul <a id='header_style'></a>

In [39]:
# list nama kolom pada tabel df
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

Kita akan mengubah nama kolom sesuai dengan aturan gaya penulisan yang baik:
* Jika nama memiliki beberapa kata, gunakan snake_case
* Semua karakter harus menggunakan huruf kecil
* Hapus spasi

In [6]:
df.columns.str.replace(' ', '').str.replace('userID', 'user_id').str.lower() 
df.columns = df.columns.str.replace(' ', '').str.replace('userID', 'user_id').str.lower()

Periksa hasilnya. Tampilkan nama kolom sekali lagi:

In [7]:
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Nilai-Nilai yang Hilang <a id='missing_values'></a>

In [8]:
# menghitung nilai yang hilang
df.isna().sum()

user_id       0
track      1343
artist     7567
genre      1198
city          0
time          0
day           0
dtype: int64

Tidak semua nilai yang hilang berpengaruh terhadap penelitian. Misalnya, nilai yang hilang dalam kolom `track` dan `artist` tidak begitu penting. Namun, nilai yang hilang dalam kolom `'genre'` dapat memengaruhi perbandingan preferensi musik di Springfield dan Shelbyville. Oleh karena itu, kita harus:
* Mengisi nilai yang hilang dengan penanda
* Mengevaluasi seberapa besar nilai yang hilang dapat memengaruhi perhitungan 

Kita akan mengganti nilai yang hilang pada kolom `'track'`, `'artist'`, dan `'genre'` dengan *string* `'unknown'`. 

In [9]:
df.fillna('unknown',inplace=True)       

Pastikan tidak ada tabel lagi yang berisi nilai yang hilang. Hitung kembali nilai yang hilang.

In [10]:
# menghitung nilai yang hilang
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Duplikat <a id='duplicates'></a>

In [11]:
# menghitung duplikat eksplisit
df.duplicated().sum()

3826

In [12]:
# menghapus duplikat eksplisit
df.drop_duplicates(inplace=True)

Hitung duplikat eksplisit sekali lagi untuk memastikan bahwa Anda telah menghapus semuanya:

In [13]:
# memeriksa duplikat
df.duplicated().sum()

0

Sekarang kita akan menghapus duplikat implisit di kolom `genre`. Misalnya, nama genre dapat ditulis dengan cara yang berbeda. Kesalahan seperti ini juga akan memengaruhi hasil.

In [14]:
# melihat nama genre yang unik
df['genre'].unique()

array(['rock', 'pop', 'folk', 'dance', 'rusrap', 'ruspop', 'world',
       'electronic', 'unknown', 'alternative', 'children', 'rnb', 'hip',
       'jazz', 'postrock', 'latin', 'classical', 'metal', 'reggae',
       'triphop', 'blues', 'instrumental', 'rusrock', 'dnb', 'türk',
       'post', 'country', 'psychedelic', 'conjazz', 'indie',
       'posthardcore', 'local', 'avantgarde', 'punk', 'videogame',
       'techno', 'house', 'christmas', 'melodic', 'caucasian',
       'reggaeton', 'soundtrack', 'singer', 'ska', 'salsa', 'ambient',
       'film', 'western', 'rap', 'beats', "hard'n'heavy", 'progmetal',
       'minimal', 'tropical', 'contemporary', 'new', 'soul', 'holiday',
       'german', 'jpop', 'spiritual', 'urban', 'gospel', 'nujazz',
       'folkmetal', 'trance', 'miscellaneous', 'anime', 'hardcore',
       'progressive', 'korean', 'numetal', 'vocal', 'estrada', 'tango',
       'loungeelectronic', 'classicmetal', 'dubstep', 'club', 'deep',
       'southern', 'black', 'folkrock', 

Dari nama genre unik di atas, terlihat adanya duplikat implisit dari genre `hiphop`. Duplikat tersebut bisa berupa nama yang ditulis secara tidak tepat atau nama alternatif dari genre yang sama.

Di sini kita bisa melihat duplikat implisit berikut:
* `hip`
* `hop`
* `hip-hop`

Kita akan membuat suatu fungsi untuk mengganti duplikat implisit di atas

In [15]:
# fungsi untuk mengganti duplikat implisit
def replace_wrong_genres(wrong_values, correct_value):
    for wrong_value in wrong_values:
        df['genre'] = df['genre'].replace(wrong_value, correct_value)

In [16]:
# masukkan fungsi yang mengganti duplikat implisit
duplicates = ['hip','hop','hip-hop']
name = 'hiphop' 
replace_wrong_genres(duplicates, name) 

Pastikan bahwa nilai yang terduplikasi telah dihapus. Tampilkan *list* nilai unik dari kolom `'genre'`:

In [17]:
# memeriksa duplikat implisit
df['genre'].unique()

array(['rock', 'pop', 'folk', 'dance', 'rusrap', 'ruspop', 'world',
       'electronic', 'unknown', 'alternative', 'children', 'rnb',
       'hiphop', 'jazz', 'postrock', 'latin', 'classical', 'metal',
       'reggae', 'triphop', 'blues', 'instrumental', 'rusrock', 'dnb',
       'türk', 'post', 'country', 'psychedelic', 'conjazz', 'indie',
       'posthardcore', 'local', 'avantgarde', 'punk', 'videogame',
       'techno', 'house', 'christmas', 'melodic', 'caucasian',
       'reggaeton', 'soundtrack', 'singer', 'ska', 'salsa', 'ambient',
       'film', 'western', 'rap', 'beats', "hard'n'heavy", 'progmetal',
       'minimal', 'tropical', 'contemporary', 'new', 'soul', 'holiday',
       'german', 'jpop', 'spiritual', 'urban', 'gospel', 'nujazz',
       'folkmetal', 'trance', 'miscellaneous', 'anime', 'hardcore',
       'progressive', 'korean', 'numetal', 'vocal', 'estrada', 'tango',
       'loungeelectronic', 'classicmetal', 'dubstep', 'club', 'deep',
       'southern', 'black', 'folkrock

## Tahap 3. Pengujian Hipotesis <a id='hypotheses'></a>

### Hipotesis 1: Membandingkan Perilaku Pengguna di Dua Kota <a id='activity'></a>

Menurut hipotesis pertama, pengguna dari Springfield dan Shelbyville memiliki perbedaan perilaku dalam mendengarkan musik. Pengujian ini menggunakan data yang diambil dari tiga hari dalam seminggu: Senin, Rabu, dan Jumat.

Kita akan mengelompokkan data berdasarkan kota dan temukan jumlah trek lagu yang diputar di setiap kelompok.

In [18]:
# Menghitung trek lagu yang diputar di setiap kota
total_track_city = df.groupby('city')['track'].count()

Pengguna dari Springfield memutar lebih banyak trek lagu daripada pengguna Shelbyville. Namun, hal ini tidak mengisyaratkan bahwa warga Springfield lebih sering mendengarkan musik. Kota ini lebih besar, dan terdapat lebih banyak pengguna.

Sekarang, kita akan mengeelompokkan data berdasarkan hari dan temukan jumlah trek lagu yang diputar pada hari Senin, Rabu, dan Jumat.

In [19]:
# Menghitung trek lagu yang diputar pada masing-masing hari
total_track_days = df.groupby('day')['track'].count()

Rabu adalah hari yang paling tenang secara keseluruhan. Namun jika kita mempertimbangkan kedua kota secara terpisah, kita mungkin akan mendapatkan kesimpulan yang berbeda.

Sekarang, kita akan membuat sebuah fungsi yang akan mengelompokkan data berdasarkan kota dan hari untuk menghitung jumlah trek lagu yang diputar untuk hari dan kota tertentu.

In [20]:
def number_tracks(day,city):
    track_list = df[(df['day']== day) & (df['city']== city)].sort_values(by='user_id')
    track_list_count = track_list['user_id'].count()
    return track_list_count  
    print()

In [21]:
# jumlah lagu yang diputar di Springfield pada hari Senin
number_tracks('Monday','Springfield')

15740

In [22]:
# jumlah lagu yang diputar di Shelbyville pada hari Senin
number_tracks('Monday','Shelbyville')

5614

In [23]:
#  jumlah lagu yang diputar di Springfield pada hari Rabu
number_tracks('Wednesday','Springfield')

11056

In [24]:
#  jumlah lagu yang diputar di Shelbyville pada hari Rabu
number_tracks('Wednesday','Shelbyville')

7003

In [25]:
# jumlah lagu yang diputar di Springfield pada hari Jumat
number_tracks('Friday','Springfield')

15945

In [26]:
# jumlah lagu yang diputar di Shelbyville pada hari Jumat
number_tracks('Friday','Shelbyville')

5895

Kita akan membuat sebuah tabel sederhana untuk melihat keseluruhan hasil yang diperoleh di langkah sebelumnya
* Nama kolomnya adalah: `['city', 'monday', 'wednesday', 'friday']`
* Datanya adalah hasil dari `number_tracks()`

In [40]:
# tabel dengan hasil
number_tracks = [
                ['Springfield', 15740, 11056, 15945],
                ['Shelbyville', 5614, 7003, 5895]
]
col = ['city', 'monday', 'wednesday', 'friday']
table_city = pd.DataFrame(data=number_tracks, columns=col)
print(table_city)

          city  monday  wednesday  friday
0  Springfield   15740      11056   15945
1  Shelbyville    5614       7003    5895


**Kesimpulan**

Data yang Anda dapatkan mengungkapkan perbedaan perilaku pengguna:

* Di kota Springfield, jumlah trek lagu yang diputar mencapai puncaknya pada hari Senin dan Jumat, sedangkan pada hari Rabu terjadi penurunan aktivitas.
* Di kota Shelbyville, sebaliknya, pengguna lebih banyak mendengarkan musik pada hari Rabu.
* Aktivitas pengguna pada hari Senin dan Jumat lebih sedikit.

### Hipotesis 2: Musik di Awal dan Akhir Minggu <a id='week'></a>

Menurut hipotesis kedua, pada Senin pagi dan Jumat malam, warga Springfield mendengarkan genre musik yang berbeda dari yang dinikmati warga Shelbyville.

Dapatkan tabel (pastikan nama tabel gabungan Anda cocok dengan DataFrame yang diberikan dalam dua blok kode di bawah):
* Untuk Springfield — `spr_general`
* Untuk Shelbyville — `shel_general`

In [28]:
spr_general = df.loc[df.loc[:,'city'] == 'Springfield']

In [29]:
shel_general = df.loc[df.loc[:,'city'] == 'Shelbyville']

Kita akan membuat fungsi yang menghasilkan informasi tentang 15 genre paling populer pada hari tertentu dalam periode antara dua stempel waktu.

In [30]:
def genre_weekday(df, day, time1, time2):
    genre_df = df[df['day'] == day]
    genre_df = genre_df[genre_df['time'] <= time2]
    genre_df = genre_df[genre_df['time'] >= time1]
    genre_df_grouped = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_grouped.sort_values(ascending=False)
    return genre_df_sorted[:15]

Bandingkan hasil fungsi `genre_weekday()` untuk Springfield dan Shelbyville pada Senin pagi (dari pukul 07.00 hingga 11.00) dan pada Jumat malam (dari pukul 17:00 hingga 23:00):

In [31]:
# memanggil fungsi untuk Senin pagi di Springfield
genre_weekday(spr_general,'Monday','07:00','11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
classical      157
metal          120
jazz           100
folk            97
soundtrack      95
Name: genre, dtype: int64

In [32]:
# memanggil fungsi untuk Senin pagi di Shelbyville 
genre_weekday(shel_general,'Monday','07:00','11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
world           36
rap             32
soundtrack      31
rnb             27
metal           27
Name: genre, dtype: int64

In [33]:
# memanggil fungsi untuk Jumat malam di Springfield
genre_weekday(spr_general,'Friday','17:00','23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
classical      163
alternative    163
rusrap         142
jazz           111
unknown        110
soundtrack     105
rnb             90
metal           88
Name: genre, dtype: int64

In [34]:
# memanggil fungsi untuk Jumat malam di Shelbyville
genre_weekday(shel_general,'Friday','17:00','23:00')

genre
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
unknown         47
ruspop          47
soundtrack      40
metal           39
rap             36
Name: genre, dtype: int64

**Kesimpulan**

Setelah membandingkan 15 genre teratas pada Senin pagi, kita dapat menarik kesimpulan berikut:

* Pengguna dari Springfield dan Shelbyville mendengarkan musik dengan genre yang sama. Lima genre teratas dari kedua kota sama, hanya genre rock dan elektronik yang bertukar tempat.

* Di Springfield, jumlah nilai yang hilang ternyata sangat besar, sehingga nilai `'unknown'` berada di urutan ke-10. Ini berarti bahwa nilai-nilai yang hilang mencakup proporsi data yang cukup besar.

* Untuk hari Jumat malam, situasinya juga serupa. Genre individual cukup bervariasi, tetapi secara keseluruhan, 15 besar genre untuk kedua kota sama.

Dengan demikian, hipotesis kedua sebagian terbukti benar:
* Pengguna mendengarkan musik yang sama di awal dan akhir minggu.
* Tidak ada perbedaan yang mencolok antara Springfield dan Shelbyville. Di kedua kota tersebut, pop adalah genre yang paling populer.

### Hipotesis 3: Preferensi Genre di Springfield dan Shelbyville <a id='genre'></a>

Hipotesis: Shelbyville menyukai musik rap. Warga Springfield lebih menyukai pop.

Kita akan mengelompokkan tabel `spr_general` berdasarkan genre dan mengetahui jumlah trek lagu yang dimainkan untuk setiap genre.

In [35]:
spr_genres = spr_general.groupby('genre').count().sort_values(by='genre',ascending=False)

Tampilkan 10 baris pertama dari `spr_genres`:

In [36]:
# menampilkan 10 baris pertama dari spr_genres
spr_genres.head(10)

,user_id,track,artist,city,time,day
genre,,,,,,
ïîï,1,1,1,1,1,1
worldbeat,1,1,1,1,1,1
world,1432,1432,1432,1432,1432,1432
western,64,64,64,64,64,64
vocal,68,68,68,68,68,68
videogame,66,66,66,66,66,66
vi,2,2,2,2,2,2
variété,24,24,24,24,24,24
uzbek,16,16,16,16,16,16


In [37]:
shel_genres = shel_general.groupby('genre').count().sort_values(by='genre',ascending=False)

Tampilkan 10 baris pertama dari `shel_genres`:

In [38]:
# menampilkan 10 baris pertama dari shel_genres
shel_genres.head(10)

,user_id,track,artist,city,time,day
genre,,,,,,
worldbeat,1,1,1,1,1,1
world,515,515,515,515,515,515
western,33,33,33,33,33,33
vocal,25,25,25,25,25,25
videogame,48,48,48,48,48,48
vi,1,1,1,1,1,1
variété,14,14,14,14,14,14
uzbek,12,12,12,12,12,12
urban,76,76,76,76,76,76


**Kesimpulan**

Hipotesis ini terbukti benar sebagian:
* Musik pop adalah genre yang paling populer di Springfield, seperti yang kita perkirakan.
* Namun, musik pop ternyata sama populernya baik di Springfield maupun di Shelbyville, dan musik rap ternyata tidak masuk ke daftar 5 besar genre untuk kedua kota tersebut.


# Kesimpulan Akhir <a id='end'></a>

Setelah menganalisis data yang tersedia, kita dapat menyimpulkan bahwa:

1. Aktivitas pengguna di Springfield dan Shelbyville bergantung pada harinya, walaupun kotanya berbeda.

Hipotesis pertama dapat diterima sepenuhnya.

2. Preferensi musik tidak bervariasi secara signifikan sepanjang minggu di Springfield dan Shelbyville. Kita bisa melihat perbedaan kecil dalam urutan pada hari Senin, tetapi:
* Baik di Springfield maupun di Shelbyville, pengguna paling banyak mendengarkan musik pop.

Oleh karena itu, hipotesis ini tidak dapat kita terima. 

3. Ternyata preferensi musik pengguna dari Springfield dan Shelbyville sangat mirip.

Hipotesis ketiga ditolak. 